In [1]:
import sys
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, EarlyStopping
from sklearn.model_selection import train_test_split
import time
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasAdamOptimizer
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras_vectorized import VectorizedDPKerasAdamOptimizer
from tensorflow_privacy.privacy.analysis.compute_noise_from_budget_lib import compute_noise


from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack import membership_inference_attack as mia
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackInputData
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import SlicingSpec
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackType

import tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.plotting as plotting

import numpy as np
import scipy as scp
from scipy import special

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
headers = ['age','workclass','fnlwgt','education','education-num','marital-status',
           'occupation','relationship','race','sex','capital-gain','capital-loss',
           'hours-per-week','native-country','class']
adult = pd.read_csv('../../datasets/adult.data', 
                    sep=', ', names=headers, na_values='?', engine='python')

# Drop all records with missing values
adult.dropna(inplace=True)
adult.reset_index(drop=True, inplace=True)
# Drop fnlwgt, not interesting for ML
adult.drop('fnlwgt', axis=1, inplace=True)
adult.drop('education', axis=1, inplace=True)

# Convert objects to categories
obj_columns = adult.select_dtypes(['object']).columns
adult[obj_columns] = adult[obj_columns].astype('category')

num_columns = adult.select_dtypes(['int64']).columns
adult[num_columns] = adult[num_columns].astype('float64')
for c in num_columns:
    adult[c] /= (adult[c].max()-adult[c].min())
adult['class'] = adult['class'].cat.codes

obj_columns = adult.select_dtypes(['category']).columns

adult.replace(['Divorced', 
               'Married-AF-spouse', 
               'Married-civ-spouse', 
               'Married-spouse-absent',
               'Never-married',
               'Separated',
               'Widowed'
              ],
              ['not married',
               'married',
               'married',
               'married',
               'not married',
               'not married',
               'not married'
              ], inplace = True)

adult = pd.get_dummies(adult, columns=obj_columns)
X = np.array(adult.drop('class', axis=1))
y = np.array(adult['class'])
y = np.eye(2)[y]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [3]:
learning_rate = 15e-5

In [4]:
def membership_inference_attack(model, X_train, X_test, y_train, y_test):
    print('Predict on train...')
    logits_train = model.predict(X_train, batch_size=batch_size)
    print('Predict on test...')
    logits_test = model.predict(X_test, batch_size=batch_size)

    print('Apply softmax to get probabilities from logits...')
    prob_train = special.softmax(logits_train, axis=1)
    prob_test = special.softmax(logits_test, axis=1)

    print('Compute losses...')
    cce = tf.keras.backend.categorical_crossentropy
    constant = tf.keras.backend.constant

    loss_train = cce(constant(y_train), constant(prob_train), from_logits=False).numpy()
    loss_test = cce(constant(y_test), constant(prob_test), from_logits=False).numpy()
    
    labels_train = np.argmax(y_train, axis=1)
    labels_test = np.argmax(y_test, axis=1)

    input = AttackInputData(
      logits_train = logits_train,
      logits_test = logits_test,
      loss_train = loss_train,
      loss_test = loss_test,
      labels_train = labels_train,
      labels_test = labels_test
    )

    # Run several attacks for different data slices
    attacks_result = mia.run_attacks(input,
                                     SlicingSpec(
                                         entire_dataset = True,
                                         by_class = True,
                                         by_classification_correctness = True
                                     ),
                                     attack_types = [
                                         AttackType.THRESHOLD_ATTACK,
                                         AttackType.LOGISTIC_REGRESSION,
                                         AttackType.MULTI_LAYERED_PERCEPTRON,
                                         AttackType.RANDOM_FOREST, 
                                         AttackType.K_NEAREST_NEIGHBORS,
                                         AttackType.THRESHOLD_ENTROPY_ATTACK
                                     ])

    # Plot the ROC curve of the best classifier
#     fig = plotting.plot_roc_curve(
#         attacks_result.get_result_with_max_auc().roc_curve)

    # Print a user-friendly summary of the attacks
    print(attacks_result.summary(by_slices = True))
    time.sleep(5)
    return attacks_result.get_result_with_max_auc().get_auc(), attacks_result.get_result_with_max_attacker_advantage().get_attacker_advantage()

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [5]:
def create_nn(dropout=None, regularizer=None):
    input_data = Input(shape = X_train[0].shape)
    x = Dense(40, activation='relu', kernel_regularizer=regularizer)(input_data)
    if dropout is not None:
        x = Dropout(dropout)(x)
    x = Dense(40, activation='relu', kernel_regularizer=regularizer)(x)
    if dropout is not None:
        x = Dropout(dropout)(x)
    output = Dense(2, kernel_regularizer=regularizer)(x)

    model = Model(input_data, output)
    
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
            
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    return model

def create_dp_nn(noise_multiplier, l2_norm_clip, microbatches):
    input_data = Input(shape = X_train[0].shape)
    x = Dense(40, activation='relu')(input_data)
    x = Dense(40, activation='relu')(x)
    output = Dense(2)(x)

    model = Model(input_data, output)
    
    optimizer = DPKerasAdamOptimizer(
                            l2_norm_clip=l2_norm_clip,
                            noise_multiplier=noise_multiplier,
                            num_microbatches=microbatches,
                            learning_rate=learning_rate)
    
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)
    
    model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])
    
    return model

In [6]:
epochs = 50
batch_size = 48
attacks = 1
settings = [
    (None,None),
    (0.25,None),
    (0.50,None),
    (0.75,None),
    (None,'l2'),
    (0.25,'l2'),
    (0.50,'l2'),
    (0.75,'l2'),
]
results_summary = []

for drop, reg in settings:
    # Instantiate network
    model = create_nn(dropout=drop, regularizer=reg)
    
    # Train network until convergence
    start_time = time.time()
    r = model.fit(X_train, 
                y_train, 
                validation_data=(X_test, y_test),
                epochs=epochs, 
                batch_size=batch_size
               )
    end_time = time.time()
    time_elapsed = (end_time - start_time)

    # MIA 
    aauc = []
    aadv = []
    for _ in range(attacks):
        auc, adv = membership_inference_attack(model, X_train, X_test, y_train, y_test)
        time.sleep(5)
        aauc.append(auc)
        aadv.append(adv)
    mauc = sum(aauc) / attacks
    madv = sum(aadv) / attacks

    # Write result summary
    summ = ', '.join(map(str,[
        len(r.history['loss']), #epochs
        drop,
        reg,
        r.history['loss'][-1], 
        r.history['val_loss'][-1],
        r.history['accuracy'][-1],
        r.history['val_accuracy'][-1],
        time_elapsed,
        mauc,
        madv
    ]))

    results_summary.append(summ)
    print('='*40)
    
    
print('Epochs, Dropout, Regularizer, Loss, Val loss, Accuracy, Val accuracy, Time, AUC, Advantage')
for r in results_summary:
    print(r)

Epoch 1/50
421/421 [==============================] - 3s 3ms/step - loss: 0.4018 - accuracy: 0.8115 - val_loss: 0.3726 - val_accuracy: 0.8219
Epoch 2/50
421/421 [==============================] - 1s 2ms/step - loss: 0.3506 - accuracy: 0.8353 - val_loss: 0.3471 - val_accuracy: 0.8344
Epoch 3/50
421/421 [==============================] - 1s 2ms/step - loss: 0.3352 - accuracy: 0.8437 - val_loss: 0.3386 - val_accuracy: 0.8419
Epoch 4/50
421/421 [==============================] - 1s 2ms/step - loss: 0.3290 - accuracy: 0.8471 - val_loss: 0.3357 - val_accuracy: 0.8438
Epoch 5/50
280/421 [==================>...........] - ETA: 0s - loss: 0.3208 - accuracy: 0.8514

KeyboardInterrupt: 

In [17]:
# TEST FOR DIFFERENT EPS
results_summary = []

n = X_train.shape[0]
epochs = 50
batch_size = 48
microbatches = 48
epsilons = [0.1,0.5,1,2,4,8,16,100,1000]
delta = 1e-6
min_noise = 1e-100
l2_norm_clip = 2.5
sampling_rate = batch_size / n
attacks = 1

for e in epsilons:
    # Compute noise multiplier from target epsilon
    noise_multiplier = compute_noise(n, batch_size, e, epochs, delta, min_noise)
    
    # Compute epsilon
    orders = [1 + x / 10. for x in range(1, 100)] + list(range(11, 101))
    sampling_probability = batch_size / n
    rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=noise_multiplier,
                    steps=epochs * n // batch_size,
                    orders=orders)
    eps = get_privacy_spent(orders, rdp, target_delta=delta)

    # Instantiate network
    model = create_dp_nn(noise_multiplier, l2_norm_clip, microbatches)

    # Train network
    start_time = time.time()
    r = model.fit(X_train, 
                 y_train, 
                 validation_data=(X_test, y_test), 
                 epochs=epochs, 
                 batch_size=batch_size,
                 #callbacks=[callback]
                )
    end_time = time.time()
    time_elapsed = (end_time - start_time)

    # MIA 
    aauc = []
    aadv = []
    for _ in range(attacks):
        auc, adv = membership_inference_attack(model, X_train, X_test, y_train, y_test)
        aauc.append(auc)
        aadv.append(adv)
    mauc = sum(aauc) / attacks
    madv = sum(aadv) / attacks

    # Write result summary
    summ = ', '.join(map(str,[
          len(r.history['loss']),
          e,
          delta,
          l2_norm_clip,
          noise_multiplier,
          sampling_rate,
          eps[0],
          r.history['loss'][-1], 
          r.history['val_loss'][-1],
          r.history['accuracy'][-1],
          r.history['val_accuracy'][-1],
          time_elapsed,
          mauc,
          madv
    ]))
    results_summary.append(summ)
    print('='*40)

    
print('Epochs, Target epsilon, delta, C, Sigma, Sampling rate, Epsilon, Loss, Val loss, Accuracy, Val accuracy, Time, AUC, Advantage')
for r in results_summary:
    print(r)

DP-SGD with sampling rate = 0.238% and noise_multiplier = 17994.001391472444 iterated over 21050 steps satisfies differential privacy with eps = 0.01 and delta = 1e-06.
Epoch 1/50
421/421 [==============================] - 16s 36ms/step - loss: 0.7564 - accuracy: 0.2844 - val_loss: 0.7583 - val_accuracy: 0.2768
Epoch 2/50
421/421 [==============================] - 15s 36ms/step - loss: 0.7601 - accuracy: 0.2799 - val_loss: 0.7614 - val_accuracy: 0.2755
Epoch 3/50
421/421 [==============================] - 15s 36ms/step - loss: 0.7569 - accuracy: 0.2869 - val_loss: 0.7544 - val_accuracy: 0.2822
Epoch 4/50
421/421 [==============================] - 15s 37ms/step - loss: 0.7568 - accuracy: 0.2831 - val_loss: 0.7591 - val_accuracy: 0.2756
Epoch 5/50
421/421 [==============================] - 15s 36ms/step - loss: 0.7555 - accuracy: 0.2863 - val_loss: 0.7536 - val_accuracy: 0.2813
Epoch 6/50
421/421 [==============================] - 15s 37ms/step - loss: 0.7546 - accuracy: 0.2881 - val_los

DP-SGD with sampling rate = 0.238% and noise_multiplier = 14.309830337352679 iterated over 21050 steps satisfies differential privacy with eps = 0.1 and delta = 1e-06.
Epoch 1/50
421/421 [==============================] - 16s 37ms/step - loss: 0.6994 - accuracy: 0.4622 - val_loss: 0.6759 - val_accuracy: 0.6319
Epoch 2/50
421/421 [==============================] - 15s 36ms/step - loss: 0.6547 - accuracy: 0.7096 - val_loss: 0.6353 - val_accuracy: 0.7449
Epoch 3/50
421/421 [==============================] - 15s 36ms/step - loss: 0.6208 - accuracy: 0.7462 - val_loss: 0.6080 - val_accuracy: 0.7532
Epoch 4/50
421/421 [==============================] - 16s 37ms/step - loss: 0.5983 - accuracy: 0.7494 - val_loss: 0.5862 - val_accuracy: 0.7537
Epoch 5/50
421/421 [==============================] - 15s 36ms/step - loss: 0.5808 - accuracy: 0.7498 - val_loss: 0.5741 - val_accuracy: 0.7536
Epoch 6/50
421/421 [==============================] - 15s 37ms/step - loss: 0.5695 - accuracy: 0.7499 - val_loss

DP-SGD with sampling rate = 0.238% and noise_multiplier = 3.0864306316599435 iterated over 21050 steps satisfies differential privacy with eps = 0.5 and delta = 1e-06.
Epoch 1/50
421/421 [==============================] - 21s 49ms/step - loss: 0.6241 - accuracy: 0.6776 - val_loss: 0.5667 - val_accuracy: 0.7483
Epoch 2/50
421/421 [==============================] - 22s 53ms/step - loss: 0.5540 - accuracy: 0.7487 - val_loss: 0.5462 - val_accuracy: 0.7531
Epoch 3/50
421/421 [==============================] - 21s 51ms/step - loss: 0.5483 - accuracy: 0.7498 - val_loss: 0.5454 - val_accuracy: 0.7536
Epoch 4/50
421/421 [==============================] - 21s 49ms/step - loss: 0.5472 - accuracy: 0.7500 - val_loss: 0.5442 - val_accuracy: 0.7535
Epoch 5/50
421/421 [==============================] - 20s 48ms/step - loss: 0.5464 - accuracy: 0.7500 - val_loss: 0.5416 - val_accuracy: 0.7535
Epoch 6/50
421/421 [==============================] - 18s 42ms/step - loss: 0.5404 - accuracy: 0.7500 - val_loss

DP-SGD with sampling rate = 0.238% and noise_multiplier = 1.7217127690788603 iterated over 21050 steps satisfies differential privacy with eps = 1 and delta = 1e-06.
Epoch 1/50
421/421 [==============================] - 16s 36ms/step - loss: 0.5427 - accuracy: 0.7499 - val_loss: 0.5281 - val_accuracy: 0.7536
Epoch 2/50
421/421 [==============================] - 15s 35ms/step - loss: 0.5278 - accuracy: 0.7499 - val_loss: 0.5217 - val_accuracy: 0.7536
Epoch 3/50
421/421 [==============================] - 15s 35ms/step - loss: 0.5187 - accuracy: 0.7499 - val_loss: 0.5097 - val_accuracy: 0.7536
Epoch 4/50
421/421 [==============================] - 15s 35ms/step - loss: 0.5059 - accuracy: 0.7499 - val_loss: 0.4976 - val_accuracy: 0.7536
Epoch 5/50
421/421 [==============================] - 15s 35ms/step - loss: 0.4929 - accuracy: 0.7499 - val_loss: 0.4879 - val_accuracy: 0.7536
Epoch 6/50
421/421 [==============================] - 15s 36ms/step - loss: 0.4837 - accuracy: 0.7499 - val_loss: 

DP-SGD with sampling rate = 0.238% and noise_multiplier = 1.0674083630051678 iterated over 21050 steps satisfies differential privacy with eps = 2 and delta = 1e-06.
Epoch 1/50
421/421 [==============================] - 16s 37ms/step - loss: 0.5917 - accuracy: 0.6977 - val_loss: 0.5255 - val_accuracy: 0.7536
Epoch 2/50
421/421 [==============================] - 15s 37ms/step - loss: 0.5242 - accuracy: 0.7499 - val_loss: 0.5139 - val_accuracy: 0.7536
Epoch 3/50
421/421 [==============================] - 15s 37ms/step - loss: 0.5120 - accuracy: 0.7499 - val_loss: 0.4990 - val_accuracy: 0.7536
Epoch 4/50
421/421 [==============================] - 16s 37ms/step - loss: 0.4966 - accuracy: 0.7499 - val_loss: 0.4869 - val_accuracy: 0.7536
Epoch 5/50
421/421 [==============================] - 15s 37ms/step - loss: 0.4843 - accuracy: 0.7498 - val_loss: 0.4791 - val_accuracy: 0.7536
Epoch 6/50
421/421 [==============================] - 16s 38ms/step - loss: 0.4775 - accuracy: 0.7500 - val_loss: 

...

DP-SGD with sampling rate = 0.238% and noise_multiplier = 0.6223068197048801 iterated over 21050 steps satisfies differential privacy with eps = 8 and delta = 1e-06.
Epoch 1/50
421/421 [==============================] - 18s 41ms/step - loss: 0.5760 - accuracy: 0.7425 - val_loss: 0.5330 - val_accuracy: 0.7536
Epoch 2/50
421/421 [==============================] - 17s 40ms/step - loss: 0.5190 - accuracy: 0.7499 - val_loss: 0.4972 - val_accuracy: 0.7536
Epoch 3/50
421/421 [==============================] - 17s 40ms/step - loss: 0.4860 - accuracy: 0.7499 - val_loss: 0.4804 - val_accuracy: 0.7536
Epoch 4/50
421/421 [==============================] - 17s 41ms/step - loss: 0.4722 - accuracy: 0.7499 - val_loss: 0.4665 - val_accuracy: 0.7536
Epoch 5/50
421/421 [==============================] - 17s 40ms/step - loss: 0.4644 - accuracy: 0.7499 - val_loss: 0.4667 - val_accuracy: 0.7536
Epoch 6/50
421/421 [==============================] - 17s 40ms/step - loss: 0.4629 - accuracy: 0.7501 - val_loss: 

DP-SGD with sampling rate = 0.238% and noise_multiplier = 0.5172600554247768 iterated over 21050 steps satisfies differential privacy with eps = 16 and delta = 1e-06.
Epoch 1/50
421/421 [==============================] - 17s 38ms/step - loss: 0.5420 - accuracy: 0.7381 - val_loss: 0.5018 - val_accuracy: 0.7536
Epoch 2/50
421/421 [==============================] - 16s 38ms/step - loss: 0.4960 - accuracy: 0.7499 - val_loss: 0.4839 - val_accuracy: 0.7538
Epoch 3/50
421/421 [==============================] - 16s 38ms/step - loss: 0.4782 - accuracy: 0.7499 - val_loss: 0.4753 - val_accuracy: 0.7539
Epoch 4/50
421/421 [==============================] - 16s 37ms/step - loss: 0.4694 - accuracy: 0.7535 - val_loss: 0.4677 - val_accuracy: 0.7585
Epoch 5/50
421/421 [==============================] - 16s 38ms/step - loss: 0.4590 - accuracy: 0.7604 - val_loss: 0.4593 - val_accuracy: 0.7639
Epoch 6/50
421/421 [==============================] - 16s 38ms/step - loss: 0.4490 - accuracy: 0.7794 - val_loss:

DP-SGD with sampling rate = 0.238% and noise_multiplier = 0.3320461699705749 iterated over 21050 steps satisfies differential privacy with eps = 100 and delta = 1e-06.
Epoch 1/50
421/421 [==============================] - 16s 36ms/step - loss: 0.5780 - accuracy: 0.6835 - val_loss: 0.5067 - val_accuracy: 0.7535
Epoch 2/50
421/421 [==============================] - 15s 36ms/step - loss: 0.4927 - accuracy: 0.7504 - val_loss: 0.4803 - val_accuracy: 0.7542
Epoch 3/50
421/421 [==============================] - 15s 36ms/step - loss: 0.4692 - accuracy: 0.7544 - val_loss: 0.4497 - val_accuracy: 0.7656
Epoch 4/50
421/421 [==============================] - 15s 35ms/step - loss: 0.4479 - accuracy: 0.7680 - val_loss: 0.4524 - val_accuracy: 0.7770
Epoch 5/50
421/421 [==============================] - 15s 35ms/step - loss: 0.4397 - accuracy: 0.7891 - val_loss: 0.4343 - val_accuracy: 0.7962
Epoch 6/50
421/421 [==============================] - 15s 36ms/step - loss: 0.4282 - accuracy: 0.8085 - val_loss

DP-SGD with sampling rate = 0.238% and noise_multiplier = 0.21836654222821442 iterated over 21050 steps satisfies differential privacy with eps = 1e+03 and delta = 1e-06.
Epoch 1/50
421/421 [==============================] - 17s 40ms/step - loss: 0.5589 - accuracy: 0.7027 - val_loss: 0.4776 - val_accuracy: 0.7538
Epoch 2/50
421/421 [==============================] - 17s 40ms/step - loss: 0.4647 - accuracy: 0.7553 - val_loss: 0.4630 - val_accuracy: 0.7646
Epoch 3/50
421/421 [==============================] - 17s 41ms/step - loss: 0.4527 - accuracy: 0.7745 - val_loss: 0.4504 - val_accuracy: 0.7872
Epoch 4/50
421/421 [==============================] - 17s 40ms/step - loss: 0.4412 - accuracy: 0.7991 - val_loss: 0.4474 - val_accuracy: 0.8076
Epoch 5/50
421/421 [==============================] - 17s 40ms/step - loss: 0.4362 - accuracy: 0.8129 - val_loss: 0.4511 - val_accuracy: 0.8077
Epoch 6/50
421/421 [==============================] - 17s 40ms/step - loss: 0.4391 - accuracy: 0.8145 - val_l

Epochs, Target epsilon, delta, C, Sigma, Sampling rate, Epsilon, Loss, Val loss, Accuracy, Val accuracy, Time, AUC, Advantage
50, 0.01, 1e-06, 2.5, 17994.001391472444, 0.0023752969121140144, 0, 0.7798415422439575, 0.7803100943565369, 0.2673198878765106, 0.26029735803604126, 759.0008347034454, 0.5253518123667378, 0.07423043609949465
50, 0.1, 1e-06, 2.5, 14.309830337352679, 0.0023752969121140144, 0.11208687744399892, 0.4863673448562622, 0.4875877797603607, 0.7498515248298645, 0.7535663843154907, 774.4385318756104, 0.5437999160427438, 0.08373142180019238
50, 0.5, 1e-06, 2.5, 3.0864306316599435, 0.0023752969121140144, 0.5000000000000254, 0.44302308559417725, 0.4543653428554535, 0.8076504468917847, 0.8016877770423889, 784.9998731613159, 0.5400670110288985, 0.07836430677254846
50, 1, 1e-06, 2.5, 1.7217127690788603, 0.0023752969121140144, 0.9999999999996306, 0.4501935541629791, 0.46728575229644775, 0.8186856508255005, 0.8090214729309082, 741.9407825469971, 0.5186605629387165, 0.08210010042988

In [73]:
noise_multiplier = compute_noise(n, batch_size, 1000, epochs, delta, min_noise)
orders = np.arange(1.1,10,0.1) + np.arange()
sampling_probability = batch_size / n
rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=noise_multiplier,
                    steps=epochs * n // batch_size,
                    orders=orders)
get_privacy_spent(orders, rdp, target_delta=delta)

DP-SGD with sampling rate = 0.238% and noise_multiplier = 0.21836654222821442 iterated over 21050 steps satisfies differential privacy with eps = 1e+03 and delta = 1e-06.


ValueError: math domain error

In [81]:
orders = list(np.arange(1.1,10,0.1))
orders

[1.1,
 1.2000000000000002,
 1.3000000000000003,
 1.4000000000000004,
 1.5000000000000004,
 1.6000000000000005,
 1.7000000000000006,
 1.8000000000000007,
 1.9000000000000008,
 2.000000000000001,
 2.100000000000001,
 2.200000000000001,
 2.300000000000001,
 2.4000000000000012,
 2.5000000000000013,
 2.6000000000000014,
 2.7000000000000015,
 2.8000000000000016,
 2.9000000000000017,
 3.0000000000000018,
 3.100000000000002,
 3.200000000000002,
 3.300000000000002,
 3.400000000000002,
 3.500000000000002,
 3.6000000000000023,
 3.7000000000000024,
 3.8000000000000025,
 3.9000000000000026,
 4.000000000000003,
 4.100000000000003,
 4.200000000000003,
 4.3000000000000025,
 4.400000000000003,
 4.5000000000000036,
 4.600000000000003,
 4.700000000000003,
 4.800000000000003,
 4.900000000000004,
 5.0000000000000036,
 5.100000000000003,
 5.200000000000003,
 5.300000000000004,
 5.400000000000004,
 5.5000000000000036,
 5.600000000000003,
 5.700000000000005,
 5.800000000000004,
 5.900000000000004,
 6.00000000

In [ ]:
i = 10
1ei